### 0. Install and import packages


pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 yfinance pandas numpy matplotlib requests python-dotenv


In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [8]:
from crewai import Agent, Task, Crew